In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Especifica el directorio que contiene los archivos CSV
directory = '.'  # Cambia esto al path de tu carpeta si es necesario

# Obtener la lista de todos los archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Leer cada archivo CSV con el delimitador '|'

separador = ','

df_list = [pd.read_csv(os.path.join(directory, file), sep=separador) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Mostrar el DataFrame combinado
print(combined_df)
#combined_df = combined_df.sample(frac=0.25, random_state=42)


               ts   uid  id.orig_h  id.orig_p  id.resp_h  id.resp_p  \
0    1.532523e+09   126         14        358        843        123   
1    1.532523e+09  1497         14        358        863        182   
2    1.532523e+09   407         14        358         55         23   
3    1.532523e+09   555         14        358        784        145   
4    1.532524e+09   877         14        358        908        101   
..            ...   ...        ...        ...        ...        ...   
195  1.532518e+09   781         14        358        918         23   
196  1.532518e+09  1089         14        358        840         23   
197  1.532518e+09   937         14        564        485          3   
198  1.532518e+09  1088         14        411        485          3   
199  1.532518e+09   881         14        442        485          3   

     conn_state  orig_pkts  orig_ip_bytes  resp_pkts  resp_ip_bytes    tcp  \
0             3          5            547          3            762  

In [ ]:
combined_df.replace('-', pd.NA, inplace=True)
print(combined_df.head)

<bound method NDFrame.head of                ts   uid  id.orig_h  id.orig_p  id.resp_h  id.resp_p  \
0    1.532523e+09   126         14        358        843        123   
1    1.532523e+09  1497         14        358        863        182   
2    1.532523e+09   407         14        358         55         23   
3    1.532523e+09   555         14        358        784        145   
4    1.532524e+09   877         14        358        908        101   
..            ...   ...        ...        ...        ...        ...   
195  1.532518e+09   781         14        358        918         23   
196  1.532518e+09  1089         14        358        840         23   
197  1.532518e+09   937         14        564        485          3   
198  1.532518e+09  1088         14        411        485          3   
199  1.532518e+09   881         14        442        485          3   

     conn_state  orig_pkts  orig_ip_bytes  resp_pkts  resp_ip_bytes    tcp  \
0             3          5            5

In [ ]:
label_encoder = preprocessing.LabelEncoder()

df = combined_df

if separador == ',':
  # Capturado
  #   df['ts'] = pd.to_datetime(df['ts'])
  #   df['ts'] = df['ts'].astype('int64') // 1e9

  # Prueba
    df['ts'] = pd.to_numeric(df['ts'])


df['uid']= label_encoder.fit_transform(df['uid'])
df['id.orig_h']= label_encoder.fit_transform(df['id.orig_h'])
df['id.resp_h']= label_encoder.fit_transform(df['id.resp_h'])
df['id.resp_p']= label_encoder.fit_transform(df['id.resp_p'])
df['id.orig_p']= label_encoder.fit_transform(df['id.orig_p'])



df['conn_state']= label_encoder.fit_transform(df['conn_state'])
df['orig_pkts'] = pd.to_numeric(df['orig_pkts'])
df['orig_ip_bytes'] = pd.to_numeric(df['orig_ip_bytes'])
df['resp_pkts'] = pd.to_numeric(df['resp_pkts'])
df['resp_ip_bytes'] = pd.to_numeric(df['resp_ip_bytes'])


In [ ]:
df.count()

,0
ts,200
uid,200
id.orig_h,200
id.orig_p,200
id.resp_h,200
id.resp_p,200
conn_state,200
orig_pkts,200
orig_ip_bytes,200
resp_pkts,200


In [ ]:
columns_to_keep = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
                   'conn_state', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
                   'resp_ip_bytes', 'tcp', 'udp']

df = df[columns_to_keep]
df.head()

X_test = df



In [ ]:
X_test.count()

,0
ts,200
uid,200
id.orig_h,200
id.orig_p,200
id.resp_h,200
id.resp_p,200
conn_state,200
orig_pkts,200
orig_ip_bytes,200
resp_pkts,200


In [ ]:
from sklearn.preprocessing import Normalizer

X_test.count()
X_test = X_test.dropna()

scaler = Normalizer()
X_test_scaled = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Ruta donde se guardará el modelo
model_path = '/content/drive/My Drive/Colab Notebooks/final.pkl'

# Cargar el modelo
svm_model = joblib.load(model_path)

In [ ]:
y_pred_svm = svm_model.predict(X_test)

In [ ]:
# Agregar las predicciones al DataFrame
X_test['Predicción_SVM'] = y_pred_svm

In [ ]:
# Contar la cantidad de 1s y 0s en las predicciones del modelo SVM
conteo_svm = X_test['Predicción_SVM'].value_counts()
print("Conteo de predicciones SVM:")
print(conteo_svm)


Conteo de predicciones SVM:
Predicción_SVM
1    104
0     96
Name: count, dtype: int64


In [ ]:
if conteo_svm.get(1, 0) > 0:
    paquetes_maliciosos = X_test[X_test['Predicción_SVM'] == 1]
    paquetes_maliciosos.to_csv('paquetes_maliciosos.csv', index=False)
    print("Existen paquetes que han sido considerados como maliciosos y se ha guardado el listado paquetes_maliciosos.csv")
else:
    print("Todos los paquetes se consideran normales y sin riesgo alguno.")

Existen paquetes que han sido considerados como maliciosos y se ha guardado el listado paquetes_maliciosos.csv
